In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
data = pd.read_csv('kicksharing_drop.csv', index_col=0) #считываю очищенный файл
data['local_book_start_dttm'] = pd.to_datetime(data['local_book_start_dttm'])
data['book_end_dttm'] = pd.to_datetime(data['book_end_dttm'])
data['book_start_dttm'] = pd.to_datetime(data['book_start_dttm'])
data['created_dttm'] = pd.to_datetime(data['created_dttm'])
data['hour'] = pd.to_datetime(data['local_book_start_dttm']).dt.hour

data['hour'] = np.where(data['hour'] <= 20,
                                  data['hour'] + 3,      # Условие hour <= 20
                                  (data['hour'] + 3) - 23)  # Иначе

def get_day_time(hour):
    if 4 <= hour <= 11:
        return 'morning'
    elif 12 <= hour <= 16:
        return 'afternoon'
    elif 17 <= hour <= 22:
        return 'evening'
    else:
        return 'night'

data['trip_number'] = data.groupby('party_rk').cumcount() + 1

# Применяем функцию к каждому значению столбца hour
data['day_time'] = data['hour'].apply(get_day_time)

# Проверка результата

data['seconds_difference'] = (data['book_start_dttm'] - data['created_dttm']).dt.total_seconds()
data = data[data['seconds_difference'] > 0]
mean = data['seconds_difference'].mean()
std = data['seconds_difference'].std()

# Вычисление Z-оценки
z_scores = (data['seconds_difference'] - mean) / std

# Условие для фильтрации (например, выбросы за пределами 3 стандартных отклонений)
data = data[np.abs(z_scores) <= 3]

data = data[data['age'] < 69]

data['age_square'] = data['age'] ** 2


import statsmodels.api as sm

In [5]:
data['model_Max'] = (data['transport_model'] == 'Max')
data['model_E'] = (data['transport_model'] == 'E')
data['model_Urban E-Bike'] = (data['transport_model'] == 'Urban E-Bike')
data['model_ES400'] = (data['transport_model'] == 'ES400')
data['model_ES200'] = (data['transport_model'] == 'ES200')

# Теперь можно использовать эти переменные для построения регрессионной модели
# Например, выберем следующие независимые переменные:
X = data[['age_square', 'age', 'trip_number', 'model_Max', 'model_E', 'model_Urban E-Bike', 'model_ES400', 'model_ES200']]

# Зависимая переменная:
y = data['seconds_difference']

# Преобразуем данные в тип float
X = X.astype(float)
y = y.astype(float)

# Проверка на NaN и удаление пропущенных значений
X.replace([np.inf, -np.inf], np.nan, inplace=True)
y.replace([np.inf, -np.inf], np.nan, inplace=True)
X.dropna(inplace=True)
y = y.loc[X.index]  # Синхронизация индексов

# Добавление константы для модели statsmodels
X_const = sm.add_constant(X)

# Построение модели линейной регрессии
model_sm = sm.OLS(y, X_const).fit()

# Вывод RSS и результата модели
rss2 = model_sm.ssr
print("RSS:", rss2)

# Вывод полной сводки модели
print(model_sm.summary())

RSS: 4286876.695418615
                            OLS Regression Results                            
Dep. Variable:     seconds_difference   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     108.3
Date:                Tue, 17 Dec 2024   Prob (F-statistic):          1.75e-181
Time:                        13:03:09   Log-Likelihood:            -1.0227e+06
No. Observations:              390877   AIC:                         2.045e+06
Df Residuals:                  390868   BIC:                         2.045e+06
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const        

In [6]:
data[data['gender_cd'] == "F"]['model_Max'] = (data[data['gender_cd'] == "F"]['transport_model'] == 'Max')
data[data['gender_cd'] == "F"]['model_E'] = (data[data['gender_cd'] == "F"]['transport_model'] == 'E')
data[data['gender_cd'] == "F"]['model_Urban E-Bike'] = (data[data['gender_cd'] == "F"]['transport_model'] == 'Urban E-Bike')
data[data['gender_cd'] == "F"]['model_ES400'] = (data[data['gender_cd'] == "F"]['transport_model'] == 'ES400')
data[data['gender_cd'] == "F"]['model_ES200'] = (data[data['gender_cd'] == "F"]['transport_model'] == 'ES200')

# Теперь можно использовать эти переменные для построения регрессионной модели
# Например, выберем следующие независимые переменные:
X = data[data['gender_cd'] == "F"][['age_square', 'age', 'trip_number', 'model_Max', 'model_E', 'model_Urban E-Bike', 'model_ES400', 'model_ES200']]

# Зависимая переменная:
y = data[data['gender_cd'] == "F"]['seconds_difference']

# Преобразуем данные в тип float
X = X.astype(float)
y = y.astype(float)

# Проверка на NaN и удаление пропущенных значений
X.replace([np.inf, -np.inf], np.nan, inplace=True)
y.replace([np.inf, -np.inf], np.nan, inplace=True)
X.dropna(inplace=True)
y = y.loc[X.index]  # Синхронизация индексов

# Добавление константы для модели statsmodels
X_const = sm.add_constant(X)

# Построение модели линейной регрессии
model_sm = sm.OLS(y, X_const).fit()

# Вывод RSS и результата модели
rss2 = model_sm.ssr
print("RSS:", rss2)

# Вывод полной сводки модели
print(model_sm.summary())

C:\Users\darak\AppData\Local\Temp\ipykernel_19068\1760030617.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[data['gender_cd'] == "F"]['model_Max'] = (data[data['gender_cd'] == "F"]['transport_model'] == 'Max')
C:\Users\darak\AppData\Local\Temp\ipykernel_19068\1760030617.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[data['gender_cd'] == "F"]['model_E'] = (data[data['gender_cd'] == "F"]['transport_model'] == 'E')
C:\Users\darak\AppData\Local\Temp\ipykernel_19068\1760030617.py:3: SettingWi

RSS: 616127.8511674328
                            OLS Regression Results                            
Dep. Variable:     seconds_difference   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     19.32
Date:                Tue, 17 Dec 2024   Prob (F-statistic):           2.45e-29
Time:                        13:03:29   Log-Likelihood:            -1.4806e+05
No. Observations:               56687   AIC:                         2.961e+05
Df Residuals:                   56678   BIC:                         2.962e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const        

C:\Users\darak\AppData\Local\Temp\ipykernel_19068\1760030617.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[data['gender_cd'] == "F"]['model_ES200'] = (data[data['gender_cd'] == "F"]['transport_model'] == 'ES200')


In [7]:
data[data['gender_cd'] == "M"]['model_Max'] = (data[data['gender_cd'] == "M"]['transport_model'] == 'Max')
data[data['gender_cd'] == "M"]['model_E'] = (data[data['gender_cd'] == "M"]['transport_model'] == 'E')
data[data['gender_cd'] == "M"]['model_Urban E-Bike'] = (data[data['gender_cd'] == "M"]['transport_model'] == 'Urban E-Bike')
data[data['gender_cd'] == "M"]['model_ES400'] = (data[data['gender_cd'] == "M"]['transport_model'] == 'ES400')
data[data['gender_cd'] == "M"]['model_ES200'] = (data[data['gender_cd'] == "M"]['transport_model'] == 'ES200')

# Теперь можно использовать эти переменные для построения регрессионной модели
# Например, выберем следующие независимые переменные:
X = data[data['gender_cd'] == "M"][['age_square', 'age', 'trip_number', 'model_Max', 'model_E', 'model_Urban E-Bike', 'model_ES400', 'model_ES200']]

# Зависимая переменная:
y = data[data['gender_cd'] == "M"]['seconds_difference']

# Преобразуем данные в тип float
X = X.astype(float)
y = y.astype(float)

# Проверка на NaN и удаление пропущенных значений
X.replace([np.inf, -np.inf], np.nan, inplace=True)
y.replace([np.inf, -np.inf], np.nan, inplace=True)
X.dropna(inplace=True)
y = y.loc[X.index]  # Синхронизация индексов

# Добавление константы для модели statsmodels
X_const = sm.add_constant(X)

# Построение модели линейной регрессии
model_sm = sm.OLS(y, X_const).fit()

# Вывод RSS и результата модели
rss2 = model_sm.ssr
print("RSS:", rss2)

# Вывод полной сводки модели
print(model_sm.summary())

C:\Users\darak\AppData\Local\Temp\ipykernel_19068\2345096113.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[data['gender_cd'] == "M"]['model_Max'] = (data[data['gender_cd'] == "M"]['transport_model'] == 'Max')
C:\Users\darak\AppData\Local\Temp\ipykernel_19068\2345096113.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[data['gender_cd'] == "M"]['model_E'] = (data[data['gender_cd'] == "M"]['transport_model'] == 'E')
C:\Users\darak\AppData\Local\Temp\ipykernel_19068\2345096113.py:3: SettingWi

RSS: 3639909.2933932967
                            OLS Regression Results                            
Dep. Variable:     seconds_difference   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     91.19
Date:                Tue, 17 Dec 2024   Prob (F-statistic):          4.74e-152
Time:                        13:03:47   Log-Likelihood:            -8.6708e+05
No. Observations:              331286   AIC:                         1.734e+06
Df Residuals:                  331277   BIC:                         1.734e+06
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const       